In [27]:
#!/usr/bin/env python
import pandas as pd
from sodapy import Socrata

client = Socrata("data.cdc.gov", None)

# Fetch variant names, the share column, and the week_ending column from the API
results = client.get("jr58-6ysp", content_type="json", select="variant, share, week_ending")

# Convert to pandas DataFrame and extract unique prefixes
results_df = pd.DataFrame.from_records(results)
results_df['share'] = results_df['share'].astype(float) * 100  # Multiply by 100 to convert to percentage
results_df['variant_prefix'] = results_df['variant'].str.split('.').str[0]

# Compute total share for each unique prefix and week
total_shares_per_week = results_df.groupby(['variant_prefix', 'week_ending'])['share'].sum().reset_index()

# Fetch the count for each unique prefix and week combination
counts = []
for _, row in total_shares_per_week.iterrows():
    prefix = row['variant_prefix']
    week = row['week_ending']
    count_results = client.get("jr58-6ysp", content_type="json", where=f"variant LIKE '{prefix}.%' AND week_ending='{week}'", select="COUNT(variant) AS variant_count")
    counts.append({
        'variant_prefix': prefix,
        'week_ending': week,
        'count': count_results[0]["variant_count"]
    })

# Convert the counts into a pandas DataFrame
count_df = pd.DataFrame(counts)
count_df = count_df[count_df['variant_prefix'] != 'Other'].reset_index(drop=True)

# Merge count_df with total_shares_per_week on variant_prefix and week_ending
final_df = pd.merge(count_df, total_shares_per_week, on=['variant_prefix', 'week_ending'])

# Rename columns
final_df = final_df.rename(columns={'variant_prefix': 'Variant Type', 'count': 'Count'})

# Format the 'share' column to show % sign
final_df['Total Share Percentage'] = final_df['share'].map('{:.2f}%'.format)

# Drop the unformatted 'share' column
final_df.drop('share', axis=1, inplace=True)

print(final_df)



   Variant Type              week_ending Count Total Share Percentage
0             B  2023-02-25T00:00:00.000   242                  0.02%
1             B  2023-03-04T00:00:00.000   462                  0.01%
2             B  2023-03-11T00:00:00.000   198                  0.01%
3             B  2023-04-01T00:00:00.000   374                  0.00%
4             B  2023-04-08T00:00:00.000    99                  0.00%
5             B  2023-04-15T00:00:00.000   308                  0.00%
6            BA  2023-02-25T00:00:00.000  1089                  7.58%
7            BA  2023-03-04T00:00:00.000  2079                  5.08%
8            BA  2023-03-11T00:00:00.000   880                  3.50%
9            BA  2023-04-01T00:00:00.000  1650                  0.18%
10           BA  2023-04-08T00:00:00.000   462                  0.08%
11           BA  2023-04-15T00:00:00.000  1463                  0.04%
12           BF  2023-02-25T00:00:00.000   242                  1.43%
13           BF  202

In [22]:
# Convert the appropriate columns to python datetime
# results_df['week_ending'] = pd.to_datetime(results_df['week_ending'], format='%Y-%m-%dT%H:%M:%S.%f')
# results_df['creation_date'] = pd.to_datetime(results_df['creation_date'], format='%Y-%m-%dT%H:%M:%S.%f')


In [23]:
# rename chare column then convert to floats 
# results_df.rename(columns={'share' : 'Share (%)'}, inplace=True)
# results_df['Share (%)'] = results_df['Share (%)'].astype(float) * 100
# results_df.head(5)

In [24]:
# results_df.tail()

In [10]:
# results_df['variant'].nunique()

54